# Initialization

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload  
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import os
import sys
import argparse
import csv
import json
import time
import types, copy, pprint
from datetime import datetime
import numpy as np
import pandas as pd
# import utils
from utils import getPharmacophoresFromRDKit, appendSmilesFromRDKit, appendMolFromRDKit 
from utils import my_enumerator, starmap_with_kwargs_async, starmap_with_kwargs
from itertools import repeat
import more_itertools
from multiprocessing import Pool, process
from os import getpid
from rdkit.Chem import PandasTools

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=10, infstr='inf', linewidth=150, nanstr='nan')
pd.options.display.width = 180

In [4]:
time.strftime('%X %x %Z')
# datetime.now().strftime('%X.%f')

'13:57:56 11/09/23 PST'

In [5]:
input_path ="./input/"
output_path ="./output_03102023/"
prefix = 'CP' ### Target-2' , 'MOA'
prefix_lc = prefix.lower().replace('-', '_')
 
compoundMetadataInputFile       = f"{input_path}JUMP-{prefix}_compound_library.csv"
compoundMetadataOutputFile      = f"{output_path}{prefix_lc}_compound_metadata_smiles.csv"
compoundMetadataOutputFile_2    = f"{output_path}{prefix_lc}_compound_metadata_clean.csv"
compoundMetadataOutputFile_3    = f"{output_path}{prefix_lc}_compound_metadata_clean_fp.csv"
compoundPharmacophoreFile       = f"{output_path}{prefix_lc}_compound_pharmacophores_sparse.pkl"

print(f" prefix                         : {prefix}" )
print(f" compound Metadata Input File   : {compoundMetadataInputFile}")
print(f" compound Metadata w/Smiles File: {compoundMetadataOutputFile}")
print(f" compound Metadata Output File 2: {compoundMetadataOutputFile_2}")
print(f" compound Metadata Output File 3: {compoundMetadataOutputFile_3}")
print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")

 prefix                         : CP
 compound Metadata Input File   : ./input/JUMP-CP_compound_library.csv
 compound Metadata w/Smiles File: ./output_03102023/cp_compound_metadata_smiles.csv
 compound Metadata Output File 2: ./output_03102023/cp_compound_metadata_clean.csv
 compound Metadata Output File 3: ./output_03102023/cp_compound_metadata_clean_fp.csv
 compound Pharmacophore File    : ./output_03102023/cp_compound_pharmacophores_sparse.pkl


#  - Append Smiles to metadata file using RDKit - ` getSmilesFromRDKit`

In [8]:
n_rows = 999999
skip_rows = 0

df = pd.read_csv(compoundMetadataInputFile , nrows = n_rows, skiprows = skip_rows, header=0,  names=['JCP2022','InChIKey', 'InChI'], )
df.index
df.info()
df.head(2)
df.tail(2)

RangeIndex(start=0, stop=116753, step=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116753 entries, 0 to 116752
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   JCP2022   116753 non-null  object
 1   InChIKey  116752 non-null  object
 2   InChI     116752 non-null  object
dtypes: object(3)
memory usage: 2.7+ MB


,JCP2022,InChIKey,InChI
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...


,JCP2022,InChIKey,InChI
116751,JCP2022_116753,ZZZZVNLQWWXZTQ-UHFFFAOYSA-N,InChI=1S/C14H20N2O3/c1-18-13-5-3-2-4-12(13)14(...
116752,JCP2022_999999,NaN,NaN


In [ ]:
# ## Get mol and SMILES for each InChI in the input file 
rc = appendSmilesFromRDKit(df, limit = -1, verbose = False)

In [ ]:
print(f'RC: {rc}')

In [11]:
print(df.info())
print(df.head(3))
print(df.tail(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116753 entries, 0 to 116752
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   JCP2022   116753 non-null  object
 1   InChIKey  116752 non-null  object
 2   InChI     116752 non-null  object
 3   smiles    116751 non-null  object
dtypes: object(4)
memory usage: 3.6+ MB
None
          JCP2022                     InChIKey                                              InChI                                             smiles
0  JCP2022_000001  AAAHWCWPZPSPIW-UHFFFAOYSA-N  InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...  CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...
1  JCP2022_000002  AAAJHRMBUHXWLD-UHFFFAOYSA-N  InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...                            OC1=NCCCN1Cc2ccc(Cl)cc2
2  JCP2022_000003  AAALVYBICLMAMA-UHFFFAOYSA-N  InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...          OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4
               

In [15]:
print(f" compound Metadata Input File   : {compoundMetadataInputFile}")
print(f" compound Metadata Output File  : {compoundMetadataOutputFile}")

 compound Metadata Input File   : ./input/JUMP-CP_compound_library.csv
 compound Metadata Output File  : ./output_03102023/cp_compound_metadata_smiles.csv


In [13]:
df.to_csv(compoundMetadataOutputFile, index = False)

In [14]:
df1 = pd.read_csv(compoundMetadataOutputFile)
df1.info()
df1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116753 entries, 0 to 116752
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   JCP2022   116753 non-null  object
 1   InChIKey  116752 non-null  object
 2   InChI     116752 non-null  object
 3   smiles    116751 non-null  object
dtypes: object(4)
memory usage: 3.6+ MB


,JCP2022,InChIKey,InChI,smiles
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,OC1=NCCCN1Cc2ccc(Cl)cc2
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,CCCN1N=CC=C1S(=O)(=O)N2CC3CNCC2CC3
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC3N=C(c1ccc(Cl)cc1)c2cc(ccc2N4C(C)=NN...


## Read metadata file and drop rows with NaN / None

In [5]:
df = pd.read_csv(compoundMetadataOutputFile)
print(df.info())
print(df.head(3))
print(df.tail(3))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116753 entries, 0 to 116752
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   JCP2022   116753 non-null  object
 1   InChIKey  116752 non-null  object
 2   InChI     116752 non-null  object
 3   smiles    116751 non-null  object
dtypes: object(4)
memory usage: 3.6+ MB
None
          JCP2022                     InChIKey                                              InChI                                             smiles
0  JCP2022_000001  AAAHWCWPZPSPIW-UHFFFAOYSA-N  InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...  CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...
1  JCP2022_000002  AAAJHRMBUHXWLD-UHFFFAOYSA-N  InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...                            OC1=NCCCN1Cc2ccc(Cl)cc2
2  JCP2022_000003  AAALVYBICLMAMA-UHFFFAOYSA-N  InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...          OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4
               

### Remove rows with  Nan in any column

In [6]:
df[df['smiles'].isna()]
len(df)

,JCP2022,InChIKey,InChI,smiles
65738,JCP2022_065740,ORYDPOVDJJZGHQ-UHFFFAOYSA-N,InChI=1S/C7H6N4O2/c8-7-9-11(13)6-4-2-1-3-5(6)1...,NaN
116752,JCP2022_999999,NaN,NaN,NaN


116753

In [7]:
df.dropna(axis=0, subset='smiles', inplace=True)
len(df)

116751

In [8]:
df.info()
df.loc[65736:65740]
df[116749:]

<class 'pandas.core.frame.DataFrame'>
Index: 116751 entries, 0 to 116751
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   JCP2022   116751 non-null  object
 1   InChIKey  116751 non-null  object
 2   InChI     116751 non-null  object
 3   smiles    116751 non-null  object
dtypes: object(4)
memory usage: 4.5+ MB


,JCP2022,InChIKey,InChI,smiles
65736,JCP2022_065738,ORYBDSTVITXLOM-UHFFFAOYSA-N,InChI=1S/C14H12IN5O3/c1-19(2)11-7-10(16-9-5-3-...,CN(C)C2=CC(Nc1ccc(I)cc1)=C(C3=NON=C23)[N+]([O-...
65737,JCP2022_065739,ORYBLTNNDMTRNX-UHFFFAOYSA-N,InChI=1S/C25H31FN2O4/c1-25(30)11-14-28(18-23(2...,CC1(O)CCN(CC1Oc2cccc(F)c2)C(=O)c3ccc(cc3)OCCN4...
65739,JCP2022_065741,ORYHMBVHFGOAIX-UHFFFAOYSA-N,InChI=1S/C21H22N4O3S/c1-16-7-8-17(15-20(16)29(...,Cc1ccc(cc1S(=O)(=O)N2CCOCC2)C4C=CC(=Nc3ccccc3)...
65740,JCP2022_065742,ORYIJHQLVLSPRM-UHFFFAOYSA-N,InChI=1S/C19H23ClN2O2/c1-13-11-18(19(24)12-21-...,CC2=CC(C(=O)CN1CCC(O)CC1)=C(C)N2c3ccc(Cl)cc3


,JCP2022,InChIKey,InChI,smiles
116750,JCP2022_116752,ZZZZIBSVRUABIA-UHFFFAOYSA-N,InChI=1S/C19H13F6N3O2S/c1-11-26-6-5-17(27-11)1...,Cc1nccc(n1)c2cccc(c2)NS(=O)(=O)c3cc(cc(c3)C(F)...
116751,JCP2022_116753,ZZZZVNLQWWXZTQ-UHFFFAOYSA-N,InChI=1S/C14H20N2O3/c1-18-13-5-3-2-4-12(13)14(...,COc1ccccc1C(O)=NCCN2CCOCC2


### Reindex and check 

In [9]:
df.reset_index(inplace = True)
df.loc[65736:65740]
df.loc[116749:]

,index,JCP2022,InChIKey,InChI,smiles
65736,65736,JCP2022_065738,ORYBDSTVITXLOM-UHFFFAOYSA-N,InChI=1S/C14H12IN5O3/c1-19(2)11-7-10(16-9-5-3-...,CN(C)C2=CC(Nc1ccc(I)cc1)=C(C3=NON=C23)[N+]([O-...
65737,65737,JCP2022_065739,ORYBLTNNDMTRNX-UHFFFAOYSA-N,InChI=1S/C25H31FN2O4/c1-25(30)11-14-28(18-23(2...,CC1(O)CCN(CC1Oc2cccc(F)c2)C(=O)c3ccc(cc3)OCCN4...
65738,65739,JCP2022_065741,ORYHMBVHFGOAIX-UHFFFAOYSA-N,InChI=1S/C21H22N4O3S/c1-16-7-8-17(15-20(16)29(...,Cc1ccc(cc1S(=O)(=O)N2CCOCC2)C4C=CC(=Nc3ccccc3)...
65739,65740,JCP2022_065742,ORYIJHQLVLSPRM-UHFFFAOYSA-N,InChI=1S/C19H23ClN2O2/c1-13-11-18(19(24)12-21-...,CC2=CC(C(=O)CN1CCC(O)CC1)=C(C)N2c3ccc(Cl)cc3
65740,65741,JCP2022_065743,ORYLUUJGYCTHAQ-UHFFFAOYSA-N,InChI=1S/C20H21N3O3/c1-13-5-4-6-16(9-13)20-22-...,Cc1cccc(c1)C3=NOC(CN=C(O)COc2ccc(C)c(C)c2)=N3


,index,JCP2022,InChIKey,InChI,smiles
116749,116750,JCP2022_116752,ZZZZIBSVRUABIA-UHFFFAOYSA-N,InChI=1S/C19H13F6N3O2S/c1-11-26-6-5-17(27-11)1...,Cc1nccc(n1)c2cccc(c2)NS(=O)(=O)c3cc(cc(c3)C(F)...
116750,116751,JCP2022_116753,ZZZZVNLQWWXZTQ-UHFFFAOYSA-N,InChI=1S/C14H20N2O3/c1-18-13-5-3-2-4-12(13)14(...,COc1ccccc1C(O)=NCCN2CCOCC2


### Write cleaned dataframe to CSV file `./input/cp_compound_metadata_clean.csv`

In [12]:
print(f" compound Metadata Output File  : {compoundMetadataOutputFile_2}\n")

 compound Metadata Output File  : ./output/cp_compound_metadata_clean.csv



In [13]:
df.to_csv(compoundMetadataOutputFile, index = False)

# Multiparallelism - append pharmacophore fingerprints to dataframe using `appendFromPubChem`

## Read Jump-CP compound metadata file `./output/cp_compound_metadata_X.csv`

In [7]:
print(f" prefix                          : {prefix}" )
print(f" compound Metadata Input File    : {compoundMetadataInputFile}")
# print(f" compound Metadata Output File   : {compoundMetadataOutputFile}")
# print(f" compound Metadata Output File 2 : {compoundMetadataOutputFile_2}")
# print(f" compound Metadata Output File 3 : {compoundMetadataOutputFile_3}")
print(f" compound Pharmacophore File     : {compoundPharmacophoreFile}")

 prefix                          : CP
 compound Metadata Input File    : ./input/JUMP-CP_compound_library.csv
 compound Pharmacophore File     : ./output_03102023/cp_compound_pharmacophores_sparse.pkl


In [8]:
df = pd.read_csv(compoundMetadataOutputFile_2)
print(df.info())
print(df.head(3))
print(df.tail(3))
df[df['smiles'].isna()]
len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   index     116751 non-null  int64 
 1   JCP2022   116751 non-null  object
 2   InChIKey  116751 non-null  object
 3   InChI     116751 non-null  object
 4   smiles    116751 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.5+ MB
None
   index         JCP2022                     InChIKey                                              InChI                                             smiles
0      0  JCP2022_000001  AAAHWCWPZPSPIW-UHFFFAOYSA-N  InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...  CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...
1      1  JCP2022_000002  AAAJHRMBUHXWLD-UHFFFAOYSA-N  InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...                            OC1=NCCCN1Cc2ccc(Cl)cc2
2      2  JCP2022_000003  AAALVYBICLMAMA-UHFFFAOYSA-N  InChI=1S/C20H15N3O2/c24-19-15-11-17(21-

,index,JCP2022,InChIKey,InChI,smiles


116751

### Setup parameters for multiprocessing

In [9]:
nrows = 999999
chunksize = 10000
progress_int = 500
df_iterator  = pd.read_csv(compoundMetadataOutputFile_2 , nrows=nrows, header=0,  chunksize= chunksize)


# dd = next(df_iterator)
# type(dd)
# dd.info()
# dd.head(4)

idx = my_enumerator()
fn = getPharmacophoresFromRDKit
args = zip(idx, df_iterator)
kwargs = dict({'append_pharmacophore': True, 'progress_int':progress_int}) 


##  multiprocessing: 

### method 1 - calling `starmap_with_kwargs` directly 

In [12]:
 
start_time = time.perf_counter()
print(f" {datetime.now().strftime('%X.%f')} | Started ")   
pool = Pool(6)
results = starmap_with_kwargs_async( pool, fn=fn, args=args, kwargs = kwargs, processes = 4)
# result = pool.starmap_async(get_pharmacophores, enumerate(df_iterator)) 


 13:00:07.234512 | Started 
 starmap_with_kwargs_async 
 processes  : 4
 fn         : <function getPharmacophoresFromRDKit at 0x7f667236a9e0>  
 args_iter  : <class 'zip'>   
 kwargs_iter: <class 'dict'> 

 13:00:07.284045 | starmap_with_kwargs_async() | Started 


Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-9:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kbardool/miniconda3/envs/cellpainting/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/kbardool/miniconda3/envs/cellpainting/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/kbardool/miniconda3/envs/cellpainting/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/kbardool/miniconda3/envs/cellpainting/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/kbardool/miniconda3/envs/cellpainting/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    se

### method 2 - calling `starmap_with_kwargs` directly 

`starmap_with_kwargs( fn, args , kwargs , pool = None):` 

takes a pool, function, aergs and kwargs and applies pool.starmap()

In [11]:
 
start_time = time.perf_counter()
print(f" {datetime.now().strftime('%X.%f')} | Started ")   
pool = Pool(6)
# results = starmap_with_kwargs( fn=fn, args=args, kwargs = kwargs)

 13:15:40.615166 | Started 


### method 3 -  calling `pool.starmap_async()` directly 

this method requires passing the function to the starmap_async. Here we were calling ` getPharmacophoresFromRDKit()` 

In [12]:
start_time = time.perf_counter()
print(f" {datetime.now().strftime('%X.%f')} | Started ")   
pool = Pool(6)
results = pool.starmap_async(fn, enumerate(df_iterator)) 

 13:16:28.111667 | Started 
 progress_int : 250

 ------------------------------------------------------------------------- 
 13:16:28.405176 | chunk_id: 0 | process started - chunk sz: 10000  append_phar: False
 ------------------------------------------------------------------------- 

 13:16:28.410746 | chunk_id: 0 | pharmacophore_base64 column exists

 13:16:28.414189 | chunk_id: 0 | 0 rows processed  progress_int : 250

 ------------------------------------------------------------------------- 
 13:16:28.420270 | chunk_id: 1 | process started - chunk sz: 10000  append_phar: False
 ------------------------------------------------------------------------- 

 13:16:28.427575 | chunk_id: 1 | pharmacophore_base64 column exists
 progress_int : 250 13:16:28.431506 | chunk_id: 1 | 0 rows processed 

 ------------------------------------------------------------------------- 
 13:16:28.433642 | chunk_id: 2 | process started - chunk sz: 10000  append_phar: False
 ----------------------------

[13:16:28] WARNING:  Problems/mismatches: Mobile-H( Mobile-H groups: One instead of multiple, Attachment points, Number)



 13:16:29.603989 | chunk_id: 1 | 250 rows processed 
 13:16:29.627109 | chunk_id: 3 | 250 rows processed 


[13:16:29] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Locations or number, Number; Mobile-H groups: One instead of multiple, Attachment points, Number; Charge(s): Do not match; Proton balance: Does not match)



 13:16:29.723484 | chunk_id: 2 | 250 rows processed 
 13:16:29.754227 | chunk_id: 4 | 250 rows processed 
 13:16:30.202948 | chunk_id: 5 | 250 rows processed 
 13:16:30.955367 | chunk_id: 4 | 500 rows processed 


...

 13:27:32.263104 | chunk_id: 10 | 9250 rows processed 
 13:28:09.569343 | chunk_id: 0 | 5250 rows processed 
 13:28:10.900518 | chunk_id: 0 | 5500 rows processed 
 13:28:12.537308 | chunk_id: 0 | 5750 rows processed 


[13:28:14] WARNING:  Problems/mismatches: Mobile-H( Hydrogens: Number; Mobile-H groups: One instead of multiple, Attachment points, Number; Charge(s): Do not match)



 13:28:16.284944 | chunk_id: 0 | 6000 rows processed 
 13:28:17.708474 | chunk_id: 0 | 6250 rows processed 
 13:28:45.155193 | chunk_id: 0 | 6500 rows processed 
 13:28:48.124473 | chunk_id: 0 | 6750 rows processed 
 13:28:49.367818 | chunk_id: 0 | 7000 rows processed 
 13:28:51.612537 | chunk_id: 0 | 7250 rows processed 
 13:28:53.550620 | chunk_id: 0 | 7500 rows processed 
 13:28:54.101370 | chunk_id: 10 | 9500 rows processed 
 13:28:55.008706 | chunk_id: 0 | 7750 rows processed 
 13:28:55.482390 | chunk_id: 10 | 9750 rows processed 


[13:28:56] WARNING:  Problems/mismatches: Mobile-H( Mobile-H groups: Attachment points, Number)




 ------------------------------------------------------------------------- 
 Results of processing chunk 10 
 ------------------------------------------------------------------------- 
                                          Total Read :  10000
 rows with missing InChiKey in input                 :      0 
 rows with no MOL object returned from RDKit         :      0      
 rows with with valid Mol                            :  10000   
 rows with No fingerprint computed                   :      0       
 rows with with valid Mol                            :  10000   
                                               Total :      0
 ------------------------------------------------------------------------- 
 13:28:56.884304 | chunk_id: 10 | Process end   
 ------------------------------------------------------------------------- 

 13:28:57.382920 | chunk_id: 0 | 8000 rows processed 
 13:28:58.564612 | chunk_id: 0 | 8250 rows processed 
 13:28:59.720268 | chunk_id: 0 | 8500 rows process

In [13]:
results.ready()

False

In [ ]:
#     results.wait()
sleep_interval = 600
while not results.ready():
    print(f" {datetime.now().strftime('%X.%f')} | still waiting - - sleep for {sleep_interval} seconds")
    time.sleep(sleep_interval)
else:
    results = results.get()
    finish_time = time.perf_counter()
    print(f" {datetime.now().strftime('%X.%f')} | starmap_with_kwargs_async() | Results ready. . .  ")    
    print(f" {datetime.now().strftime('%X.%f')} | All processes complete")
    print(f" {datetime.now().strftime('%X.%f')} | Program finished in {finish_time-start_time:5f} seconds")  
    
print(f" {datetime.now().strftime('%X.%f')} | Process Ended ")    

 13:16:43.478385 | still waiting - - sleep for 600 seconds
 13:26:43.579088 | still waiting - - sleep for 600 seconds
 13:36:43.679477 | still waiting - - sleep for 600 seconds
 13:46:43.779989 | still waiting - - sleep for 600 seconds
 13:56:43.880458 | still waiting - - sleep for 600 seconds
 14:06:43.973121 | still waiting - - sleep for 600 seconds
 14:16:44.073439 | still waiting - - sleep for 600 seconds
 14:26:44.173835 | still waiting - - sleep for 600 seconds
 14:36:44.274234 | still waiting - - sleep for 600 seconds
 14:46:44.313107 | still waiting - - sleep for 600 seconds
 14:56:44.413522 | still waiting - - sleep for 600 seconds
 15:06:44.438798 | still waiting - - sleep for 600 seconds
 15:16:44.497425 | still waiting - - sleep for 600 seconds
 15:26:44.597833 | still waiting - - sleep for 600 seconds
 11:31:18.054244 | still waiting - - sleep for 600 seconds
 11:41:18.154867 | still waiting - - sleep for 600 seconds
 11:51:18.255598 | still waiting - - sleep for 600 secon

### check results 

In [17]:
type(results), len(results)## , len(results[0])   ## ,  len(results[0][1])

(list, 0)

In [31]:
print(results[0][0])
print(results[0][1])
print(results[0][2])

0
      index         JCP2022                     InChIKey                                              InChI                                             smiles   
0         0  JCP2022_000001  AAAHWCWPZPSPIW-UHFFFAOYSA-N  InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...  CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...  \
1         1  JCP2022_000002  AAAJHRMBUHXWLD-UHFFFAOYSA-N  InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...                            OC1=NCCCN1Cc2ccc(Cl)cc2   
2         2  JCP2022_000003  AAALVYBICLMAMA-UHFFFAOYSA-N  InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...          OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4   
3         3  JCP2022_000004  AAANUZMCJQUYNX-UHFFFAOYSA-N  InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...                 CCCN1N=CC=C1S(=O)(=O)N2CC3CNCC2CC3   
4         4  JCP2022_000005  AAAQFGUYHFJNHI-UHFFFAOYSA-N  InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...  CCN=C(O)CC3N=C(c1ccc(Cl)cc1)c2cc(ccc2N4C(C)=NN...   
...     ...             ..

## combine metadata files generated by separate processes and write to CSV `./output/cp_compound_metadata_clean.csv`

In [ ]:
print(f" compound Metadata Input File   : {compoundMetadataInputFile}")
print(f" compound Metadata Output File  : {compoundMetadataOutputFile}\n")
print(f" compound Metadata Output File 2: {compoundMetadataOutputFile_2}\n")
print(f" compound Metadata Output File 3: {compoundMetadataOutputFile_3}\n")
print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")

In [ ]:
# for i, res in enumerate(results):
#     chunk_id, df_chunk ,df_fingerprints = res    
#     mode = 'w' if chunk_id == 0 else 'a'
#     header = chunk_id == 0
#     print(f" {datetime.now().strftime('%X.%f')} | chunk_id: {chunk_id} | write to compoundDataOutputFile     | mode : {mode}  header: {header}")
#     df_chunk.to_csv(compoundMetadataOutputFile_2,
#                     index=False,  
#                     header=header, mode=mode) # Skip index column    

In [ ]:
df_metadata_all = pd.concat([i[1] for i in results ], axis = 0, ignore_index=True )

In [37]:
# df_metadata_all.drop(columns='index', inplace = True)
df_metadata_all.info()
df_metadata_all.head()
# df_fingerprints_all.loc[197:203]

df_metadata_all[df_metadata_all['smiles'].isna()]
len(df_metadata_all)
df_metadata_all[df_metadata_all['InChI'].isna()]
len(df_metadata_all)
df_metadata_all[df_metadata_all['pharmacophore_base64'].isna()]
len(df_metadata_all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   JCP2022               116751 non-null  object
 1   InChIKey              116751 non-null  object
 2   InChI                 116751 non-null  object
 3   smiles                116751 non-null  object
 4   pharmacophore_base64  116751 non-null  object
dtypes: object(5)
memory usage: 4.5+ MB


,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...,4P///wgEAABaAAAAAgAAAAAAAAAABAAEAAAAAAACBAAIAA...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,OC1=NCCCN1Cc2ccc(Cl)cc2,4P///wgEAAAXAAAABgAEAhAAFAIAAAAO0GoADEixALUAAA...
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4,4P///wgEAABeAAAAAgAAAgAAAAAMAAAAAgAADgAAAAAMBA...
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,CCCN1N=CC=C1S(=O)(=O)N2CC3CNCC2CC3,4P///wgEAAAfAAAAAAAABAIKCgAACggUCDoCFAYGBiYGJg...
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC3N=C(c1ccc(Cl)cc1)c2cc(ccc2N4C(C)=NN...,4P///wgEAABhAAAAAAAAAAIAAAAADAACAAAAABAAAgwAAA...


,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

In [38]:
print(f" compound Metadata Output File 3: {compoundMetadataOutputFile_3}\n")

 compound Metadata Output File 2: ./output/cp_compound_metadata_clean.csv



In [39]:
df_metadata_all.to_csv(compoundMetadataOutputFile_3, header = True, index = False)

## combine all fingerprint files and write to pickle `./output/cp_compound_pharmacophores.pkl`

In [ ]:
   
#     print(f" {datetime.now().strftime('%X.%f')} | chunk_id: {chunk_id} | write to compoundPharmacophoreFile  | mode : {mode}  header: {header}")
#     df_fingerprints.to_csv(compoundPharmacophoreFile,
#                     index=False,  
#                     header=header, mode=mode)     # Skip index column

In [40]:
df_fingerprints_all = pd.concat([i[2] for i in results ], axis = 0, ignore_index=True )
df_fingerprints_all.info()
df_fingerprints_all.head()
# df_fingerprints_all.loc[197:203]

df_fingerprints_all[df_fingerprints_all['pharmacophore_fp'].isna()]
len(df_fingerprints_all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   JCP2022           116751 non-null  object
 1   pharmacophore_fp  116751 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


,JCP2022,pharmacophore_fp
0,JCP2022_000001,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ..."
1,JCP2022_000002,"[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
2,JCP2022_000003,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
3,JCP2022_000004,"[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,JCP2022_000005,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


,JCP2022,pharmacophore_fp


116751

In [41]:
print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")

 compound Pharmacophore File    : ./output/cp_compound_pharmacophores.pkl


In [42]:
df_fingerprints_all.to_pickle(compoundPharmacophoreFile)

In [43]:
# df_chunk.info()
# df_fingerprints.info()
del df_fingerprints_all,   df_metadata_all

## Reread saved files and recheck them

### check - reread the written metadata CSV file `./output/cp_compound_metadata_clean.csv`

In [63]:
print(f" compound Metadata Output File 3: {compoundMetadataOutputFile_3}\n")


 compound Metadata Output File 2: ./output/cp_compound_metadata_clean.csv



In [64]:
df = pd.read_csv(compoundMetadataOutputFile_3)
df.info()
df.head(10)
# df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   JCP2022               116751 non-null  object
 1   InChIKey              116751 non-null  object
 2   InChI                 116751 non-null  object
 3   smiles                116751 non-null  object
 4   pharmacophore_base64  116751 non-null  object
dtypes: object(5)
memory usage: 4.5+ MB


,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64
0,JCP2022_000001,AAAHWCWPZPSPIW-UHFFFAOYSA-N,InChI=1S/C25H31N5O2/c1-4-23-26-14-16-30(23)24-...,CCC1=NC=CN1c2cccc(n2)C3CCCN3C(=O)c4ccc(cc4)OCC...,4P///wgEAABaAAAAAgAAAAAAAAAABAAEAAAAAAACBAAIAA...
1,JCP2022_000002,AAAJHRMBUHXWLD-UHFFFAOYSA-N,InChI=1S/C11H13ClN2O/c12-10-4-2-9(3-5-10)8-14-...,OC1=NCCCN1Cc2ccc(Cl)cc2,4P///wgEAAAXAAAABgAEAhAAFAIAAAAO0GoADEixALUAAA...
2,JCP2022_000003,AAALVYBICLMAMA-UHFFFAOYSA-N,InChI=1S/C20H15N3O2/c24-19-15-11-17(21-13-7-3-...,OC2=NC(=O)c3cc(Nc1ccccc1)c(cc23)Nc4ccccc4,4P///wgEAABeAAAAAgAAAgAAAAAMAAAAAgAADgAAAAAMBA...
3,JCP2022_000004,AAANUZMCJQUYNX-UHFFFAOYSA-N,InChI=1S/C13H22N4O2S/c1-2-7-16-13(5-6-15-16)20...,CCCN1N=CC=C1S(=O)(=O)N2CC3CNCC2CC3,4P///wgEAAAfAAAAAAAABAIKCgAACggUCDoCFAYGBiYGJg...
4,JCP2022_000005,AAAQFGUYHFJNHI-UHFFFAOYSA-N,InChI=1S/C22H22ClN5O2/c1-4-24-20(29)12-18-22-2...,CCN=C(O)CC3N=C(c1ccc(Cl)cc1)c2cc(ccc2N4C(C)=NN...,4P///wgEAABhAAAAAAAAAAIAAAAADAACAAAAABAAAgwAAA...
5,JCP2022_000006,AAAROXVLYNJINN-UHFFFAOYSA-N,"InChI=1S/C16H20N6O/c1-16(2,3)22-13(10-5-6-10)7...",CN1C=C(C=N1)C2=NOC(=N2)C4C=C(C3CC3)N(N=4)C(C)(C)C,4P///wgEAABHAAAAAAAAAAQAAAAMAAQAAAAmAg4wAAACBg...
6,JCP2022_000007,AAATVJFGQAZXQO-UHFFFAOYSA-N,InChI=1S/C24H22N2O4S/c1-14-20(15(2)30-26-14)13...,CC(OC(=O)c1ccccc1SCC2C(C)=NOC=2C)C(=O)C3=CNc4c...,4P///wgEAABiAAAAAAAAAAIAAAAADgAAAAACEAAAAAwAAA...
7,JCP2022_000008,AAAUIKUIDRZZQT-UHFFFAOYSA-N,InChI=1S/C26H24ClN3O2/c1-3-22(25(31)29-20-13-9...,CCC(C(O)=Nc1cccc(Cl)c1)N4C(=O)C(C)N=C(c2ccccc2...,4P///wgEAAA9AAAAAgICAgIMAAIAAAAAEgIMAAAADABCHg...
8,JCP2022_000009,AAAZRMGPBSWFDK-UHFFFAOYSA-N,"InChI=1S/C18H19F3N4O4S2/c19-18(20,21)13-3-1-2-...",NC(=O)C1C=CSC=1N=C(O)CN2CCN(CC2)S(=O)(=O)c3ccc...,4P///wgEAABKAAAAAAAAAAAAAAAAAAYIAA4GAAAAAAYEDA...
9,JCP2022_000010,AABLKAPUNFMSIA-UHFFFAOYSA-N,InChI=1S/C15H24N4O4S/c1-3-18-14(4-7-16-18)15(2...,CCN1N=CC=C1C(=O)N2CC(C2)N(C3CCOCC3)S(C)(=O)=O,4P///wgEAAAPAAAAAAAAAAAAGHYIBAgEAGAKxQs=


In [65]:
df[df['InChI'].isna()]
len(df)
df[df['smiles'].isna()]
len(df)
df[df['pharmacophore_base64'].isna()]
len(df)


,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

,JCP2022,InChIKey,InChI,smiles,pharmacophore_base64


116751

In [47]:
del df 

### check - reread the written fingerprint pickle file `./output/cp_compound_pharmacophores.pkl`

In [54]:
print(f" compound Pharmacophore File    : {compoundPharmacophoreFile}")

 compound Pharmacophore File    : ./output/cp_compound_pharmacophores.pkl


In [44]:
df = pd.read_pickle(compoundPharmacophoreFile)
df.info()
df.head(10)
df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   JCP2022           116751 non-null  object
 1   pharmacophore_fp  116751 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


,JCP2022,pharmacophore_fp
0,JCP2022_000001,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ..."
1,JCP2022_000002,"[0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
2,JCP2022_000003,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
3,JCP2022_000004,"[1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,JCP2022_000005,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
5,JCP2022_000006,"[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
6,JCP2022_000007,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
7,JCP2022_000008,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, ..."
8,JCP2022_000009,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ..."
9,JCP2022_000010,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


,JCP2022,pharmacophore_fp
116741,JCP2022_116744,"[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116742,JCP2022_116745,"[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116743,JCP2022_116746,"[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116744,JCP2022_116747,"[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
116745,JCP2022_116748,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116746,JCP2022_116749,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, ..."
116747,JCP2022_116750,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, ..."
116748,JCP2022_116751,"[0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116749,JCP2022_116752,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
116750,JCP2022_116753,"[0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, ..."


In [45]:

print(type(df.at[0, 'pharmacophore_fp']), len(df.at[0, 'pharmacophore_fp']),  df.at[0, 'pharmacophore_fp'])
print(type(df.loc[0].pharmacophore_fp)  , len(df.loc[0].pharmacophore_fp)  ,  df.loc[0].pharmacophore_fp)
print(type(df.iloc[0].pharmacophore_fp) , len(df.iloc[0].pharmacophore_fp) ,  df.iloc[0].pharmacophore_fp)
# print(type(df[0].pharmacophore_fp) , len(df[0].pharmacophore_fp) ,  df[0].pharmacophore_fp)


<class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 1032 <rdkit.DataStructs.cDataStructs.SparseBitVect object at 0x7fc484318f90>
<class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 1032 <rdkit.DataStructs.cDataStructs.SparseBitVect object at 0x7fc484318f90>
<class 'rdkit.DataStructs.cDataStructs.SparseBitVect'> 1032 <rdkit.DataStructs.cDataStructs.SparseBitVect object at 0x7fc484318f90>


# Figuring out what to do with the fingerprints --> Sparse CSR matrix

In [75]:
# import rdkit
# from rdkit.Chem import AllChem
# # from rdkit.Chem.Draw import rdMolDraw2D
# from rdkit.Chem import inchi

# from rdkit.Chem.Pharm2D import Generate
# import rdkit.Avalon.pyAvalonTools as pat

from rdkit import RDConfig
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory    

fdefName = os.path.join(RDConfig.RDDataDir,'BaseFeatures.fdef')
#     print(f" RDKit BaseFeatures.fdef : {fdefName}")
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory,minPointCount=2,maxPointCount=3)
sigFactory.SetBins( [(0,3),(3,8)])
sigFactory.Init()
ncols=sigFactory.GetSigSize()    
print(ncols)

1032


In [67]:
input_path ="./input/"
output_path ="./output/"
prefix = 'CP' ### Target-2' , 'MOA'
prefix_lc = prefix.lower().replace('-', '_')
 
# compoundMetadataInputFile   = f"{input_path}JUMP-{prefix}_compound_library.csv"

# compoundMetadataInputFile      = f"{output_path}{prefix_lc}_compound_metadata_clean.csv"
# compoundMetadataInputFileTPSA  = f"{input_path}{prefix_lc}_compound_metadata_clean_tpsa.csv"
# # compoundMetadataOutputFile_2   = f"{output_path}{prefix_lc}_compound_metadata_2_X.csv"
# compoundTPSAFile                = f"{output_path}{prefix_lc}_compound_TPSA.csv"
# compoundPharmacophoreFile       = f"{output_path}{prefix_lc}_compound_pharmacophores.pkl"

print(f" prefix                             : {prefix} \n" )
print(f" compound Metadata Input File       : {compoundMetadataInputFile} \n")
print(f" compound Metadata Input File TPSA  : {compoundMetadataInputFileTPSA}")
print(f" compound Metadata Output File      : {compoundMetadataOutputFile}\n")
print(f" compound TPSA File                 : {compoundTPSAFile}")
print(f" compound Pharmacophore File        : {compoundPharmacophoreFile}")

 prefix                             : CP 

 compound Metadata Input File       : ./output/cp_compound_metadata_clean.csv 

 compound Metadata Input File TPSA  : ./input/cp_compound_metadata_clean_tpsa.csv
 compound Metadata Output File      : ./output/cp_compound_metadata_clean.csv

 compound TPSA File                 : ./output/cp_compound_TPSA.csv
 compound Pharmacophore File        : ./output/cp_compound_pharmacophores.pkl


In [72]:
df = pd.read_pickle(compoundPharmacophoreFile)
print(df.info())
print(df.head(5))
# print(df.tail(10))
df.info() 
type(df.loc[0].pharmacophore_fp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   JCP2022           116751 non-null  object
 1   pharmacophore_fp  116751 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB
None
          JCP2022                                   pharmacophore_fp
0  JCP2022_000001  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, ...
1  JCP2022_000002  [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...
2  JCP2022_000003  [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
3  JCP2022_000004  [1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...
4  JCP2022_000005  [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116751 entries, 0 to 116750
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   JCP2022           116751 non-null 

rdkit.DataStructs.cDataStructs.SparseBitVect

In [ ]:
chunk_id = 0
NoMol_ctr, Found_ctr, Mol_Found_ctr = 0,0,0
NoSmiles_ctr, Smiles_Found_ctr = 0,0 
FP_ctr, NoFP_ctr = 0,0
smiles = True
mol = True
fingerprints = []
append_pharmacophore = True
pharmacophore_col_name = 'pharmacophore_fp'
FPSparse = None
fp_prev = None
np_prev = None
base64_prev = None
 
fp_fromBase64Sparse = rdkit.DataStructs.cDataStructs.SparseBitVect(0)
fp_fromBase64Explicit = rdkit.DataStructs.cDataStructs.ExplicitBitVect(0)

np_rows = []
indptr  = np.zeros((1,),dtype=np.int8)
indices = np.empty((0,),dtype=np.int8)
data    = np.empty((0,),dtype=np.int8)
print(indptr)
print(indices)
print(data)

ctr = 0
for comp in df.itertuples():
    if ctr > 20: break
    print()
    if ctr % 1 == 0:
        print(f" {datetime.now().strftime('%X.%f')} | chunk_id: {chunk_id} | {ctr} rows processed | index: {comp.Index} | {df.at[comp.Index, 'JCP2022']}", flush = True)
    ctr +=1

    ## Get Mol from InChI and generate 2D pharmacophore fingerprint
    mol_tmp = inchi.MolFromInchi(comp.InChI, sanitize = True, removeHs = True)
    
    if mol_tmp is None:
        print(f" ctr: {ctr} index: {comp.Index}   InChIKey: {comp.InChIKey } \t No MOL returned for inchi from RDKit - continue", flush = True)
        fingerprints.append((df.at[comp.Index, 'JCP2022'], np.nan))
        NoMol_ctr +=1
        continue
    else:
        Mol_Found_ctr +=1
        print(f" ctr: {ctr} index: {comp.Index}   mol: {mol_tmp}  mol type:{type(mol_tmp)} - get 2D fingerprint") 
        FPSparse = Generate.Gen2DFingerprint(mol_tmp, sigFactory)

    if FPSparse is None:
        print(f" ctr: {ctr} index: {comp.Index}   InChIKey: {comp.InChIKey } \t No Fingerprint computed from molecule file  - continue", flush = True)
        fingerprints.append((df.at[comp.Index, 'JCP2022'], np.nan))
        NoFP_ctr +=1
    else:
        base64_sparse = rdkit.DataStructs.cDataStructs.SparseBitVect.ToBase64(FPSparse)
        FP_ctr += 1 
        
        FPExplicit =rdkit.DataStructs.cDataStructs.ConvertToExplicit(FPSparse)        
        base64_explicit = rdkit.DataStructs.cDataStructs.ExplicitBitVect.ToBase64(FPExplicit)
        
#         from base64 ---> Explicit & Sparse Bit Vector 
#         rdkit.DataStructs.cDataStructs.ExplicitBitVect.FromBase64(fp_fromBase64Explicit, base64_explicit)
#         rdkit.DataStructs.cDataStructs.SparseBitVect.FromBase64(fp_fromBase64Sparse, base64_sparse)        
        
#         from FPSparse and FPExplicit ---> lists
#         list_fromFPSparse = rdkit.DataStructs.cDataStructs.SparseBitVect.ToList(FPSparse)     
#         list_fromFPExplicit = rdkit.DataStructs.cDataStructs.ExplicitBitVect.ToList(FPExplicit)
        
#         from FPSparse and FPExplicit ---> lists
#         list_fromBase64Explicit = rdkit.DataStructs.cDataStructs.ExplicitBitVect.ToList(fp_fromBase64Explicit)          
#         list_fromBase64Sparse = rdkit.DataStructs.cDataStructs.SparseBitVect.ToList(fp_fromBase64Sparse)        

#         from FPExplict, fromBase64Explicit ---> numpy arrays
#         np_fromFPExplicit = np.zeros((0), dtype=np.int8)
#         np_fromBase64Explicit= np.zeros((0), dtype=np.int8) 
#         rdkit.DataStructs.cDataStructs.ConvertToNumpyArray(FPExplicit, np_fromFPExplicit)
#         rdkit.DataStructs.cDataStructs.ConvertToNumpyArray(fp_fromBase64Explicit, np_fromBase64Explicit)

#         print(f" base64: {base64_sparse} ", flush = True)
#         print()
#         print(f" fp = fp_prev?                         : {FPSparse == fp_prev}")
#         print(f" base64_sparse = base64_prev?          : {base64_prev == base64_sparse} ")  
        print(f" base64_sparse = base64_explicit : {base64_explicit == base64_sparse} ")  
#         if np_prev is not None:  
#             print(f" np_explicit = np_prev                 : {np_fromFPExplicit.sum()}  {np_prev.sum()}    {(np_fromFPExplicit == np_prev).all()} ")
#         print(f" np_fromBase64Sparse = fp              : {fp_fromBase64Sparse == FPSparse} ")
#         print(f" np_fromBase64Explicit = fp_explcit    : {fp_fromBase64Explicit == FPExplicit} " )
#         print(f" **_list_1 = **_list_2                 : {list_fromFPExplicit == list_fromBase64Explicit}   {list_fromFPExplicit == list_fromFPSparse }   {list_fromFPSparse == list_fromBase64Sparse}" )
#         print(f" np_explicit = np_explicit_2           : {np_fromFPExplicit.dtype}  {np_fromFPExplicit.sum()}  {np_fromBase64Explicit.sum()} {(np_fromFPExplicit == np_fromBase64Explicit).all()} ")
#         base64_prev = base64_sparse
#         np_prev = np_fromFPExplicit
#         fp_prev = FPSparse


        ## Build sparse matrix representation 
    
        onbits = list(FPSparse.GetOnBits())
        len_onbits = len(onbits)            
        next_indptr = indptr[-1] + len_onbits
        indptr  = np.concatenate([indptr, [next_indptr]])
        data    = np.concatenate([data, np.ones(len_onbits)])
        indices = np.concatenate([indices,onbits])
        
#         print()
#         print(f" onbits {len(onbits):5d} : {onbits}")
#         print(f" indptr {len(indptr):5d} : {indptr}")
#         print(f" data   {  len(data):5d} : {data}")
#         print(f" indices{len(indices):5d}: {indices}")

    fingerprints.append((df.at[comp.Index,'JCP2022'], FPSparse, base64_sparse, onbits))
                                                                              
    if append_pharmacophore:
        df.at[comp.Index, "pharmacophore_fp" ]    = FPSparse
        df.at[comp.Index, "pharmacophore_base64"] = base64_sparse
        
 

from scipy import sparse
csr = sparse.csr_matrix((data, indices, indptr), shape=(len(indptr)-1, ncols), dtype = np.int8)
csr

In [ ]:
# print(csr[3])

In [ ]:
len(fingerprints[0])
fingerprints[0][0]
fingerprints[0][1]
fingerprints[0][2]
type(fingerprints[0][3]), len(fingerprints[0][3])

# for i in fingerprints:
#     print(i[0], '    ',i[3])

In [ ]:
df_fingerprints = pd.DataFrame(fingerprints, columns=['JCP2022', 'pharmacophore_fp', 'pharmacophore_base64','pharmacophore_onbits' ])        
df_fingerprints.info()
df_fingerprints.head(10)

###  write fingerprints to pickle file

In [ ]:
# df_fingerprints.to_csv(compoundPharmacophoreFile, index = False)
df_fingerprints.to_pickle('./output/cp_compound_pharmacophore_X.pkl')

### Read back from written pharmacohpore pickle file

In [ ]:
print(f" prefix                       : {prefix}" )
print(f" compound Metadata Input File  : {compoundMetadataInputFile}")
print(f" compound Metadata Output File : {compoundMetadataOutputFile}")
print(f" compound Metadata Output File 2: {compoundMetadataOutputFile_2}")
print(f" compound Pharmacophore File  : {compoundPharmacophoreFile}")

In [ ]:
df_fingerprints = pd.read_pickle('./output/cp_compound_pharmacophore_X.pkl')
# df_fingerprints = pd.read_csv(compoundPharmacophoreFile)

df_fingerprints.info()
df_fingerprints.head(10)
np_prev = None
ex_prev = None

In [ ]:

df_fingerprints.iloc[0,0]
df_fingerprints.iloc[0,1]
df_fingerprints.iloc[0,2]
type(df_fingerprints.iloc[0,3]), len(df_fingerprints.iloc[0,3])
type(df_fingerprints.loc[0].pharmacophore_onbits)
# for i in fingerprints:
#     print(i[0], '    ',i[3])

In [73]:
featFactory = ChemicalFeatures.BuildFeatureFactory(fdefName)
sigFactory = SigFactory(featFactory,minPointCount=2,maxPointCount=3)
sigFactory.SetBins( [(0,3),(3,8)])
sigFactory.Init()
ncols=sigFactory.GetSigSize()    
print(ncols)

NameError: name 'ChemicalFeatures' is not defined

In [ ]:
fp_fromBase64Sparse = rdkit.DataStructs.cDataStructs.SparseBitVect(0)
fp_fromBase64Explicit = rdkit.DataStructs.cDataStructs.ExplicitBitVect(0)
np_rows = []
indptr  = np.zeros((1,),dtype=np.int8)
indices = np.empty((0,),dtype=np.int8)
data    = np.empty((0,),dtype=np.int8)
print(indptr)
print(indices)
print(data)

In [ ]:
for comp in df_fingerprints.itertuples():
    
    rdkit.DataStructs.cDataStructs.ExplicitBitVect.FromBase64(fp_fromBase64Explicit, comp.pharmacophore_base64)
    rdkit.DataStructs.cDataStructs.SparseBitVect.FromBase64(fp_fromBase64Sparse, comp.pharmacophore_base64)
       
#     explicit_list = rdkit.DataStructs.cDataStructs.ExplicitBitVect.ToList(fp_fromBase64Explicit)
#     sparse_list = rdkit.DataStructs.cDataStructs.SparseBitVect.ToList(fp_fromBase64Sparse)        

    ## fromBase64Explicit ---> numpy arrays
#     np_fromBase64Explicit= np.zeros((0), dtype=np.int8) 
#     rdkit.DataStructs.cDataStructs.ConvertToNumpyArray(fp_fromBase64Explicit, np_fromBase64Explicit)    
    
    print(f" {comp.JCP2022}      pharmacophore_np: {type(comp.pharmacophore_fp)}  pharmacophore_base64: {type( comp.pharmacophore_base64)}   pharmacophore_onbits: {type(comp.pharmacophore_onbits)} ")
    print(f"                                                                      pharmacophore_base64: {comp.pharmacophore_base64[:7]}      pharmacophore_onbits: {comp.pharmacophore_onbits[:7]}")
    
#     print(f" expl_vect :    {type(expl_vect)}    sparse_list: {type(sparse_list)}   explicit_list: {type(explicit_list)}")
#     print(f"  sparse_list  == explicit_list ?     {sparse_list == explicit_list}")
#     print(f"  sparse_numpy == explicit_numpy?     {sparse_list == explicit_list}")
#     print(f"  sparse_numpy == explicit_numpy?     {np_fromBase64Explicit.dtype}   {np_fromBase64Explicit.sum()}  ")

    onbits = list(comp.pharmacophore_fp.GetOnBits())

    print(f" pharmacophore_onbits: {comp.pharmacophore_onbits}")    
    print(f"                     : {onbits} ")
    print()
    
    ## Build sparse matrix representation 

#     onbits = list(FPSparse.GetOnBits())
#     len_onbits = len(onbits)            
#     next_indptr = indptr[-1] + len_onbits
#     indptr  = np.concatenate([indptr, [next_indptr]])
#     data    = np.concatenate([data, np.ones(len_onbits)])
#     indices = np.concatenate([indices,onbits])
#         print()
#         print(f" onbits {len(onbits):5d} : {onbits}")
#         print(f" indptr {len(indptr):5d} : {indptr}")
#         print(f" data   {  len(data):5d} : {data}")
#         print(f" indices{len(indices):5d}: {indices}")
    
    np_prev= np_explicit
    ex_prev= expl_vect
    np_rows.append(np_fromBase64Explicit)
# inchi_1 = df.loc[0, 'InChI']
# m1 = df.loc[0, 'mol']
# m2 = df.loc[0, 'mol_2']

# print(type(inchi_1), type(m1), type(m2))
# m1
# m2

In [ ]:
len(np_rows[0])

In [ ]:
print(f" indptr {len(indptr):5d} : {indptr}")
print(f" data   {len(data):5d}   : {data}")
print(f" indices{len(indices):5d}: {indices}")

In [ ]:
from scipy import sparse
csr = sparse.csr_matrix((data, indices, indptr), shape=(len(indptr)-1, len(np_rows[0])), dtype = np.int8)

In [ ]:
csr